# Simple Time-Series Model Fit

In the Black-Scholes world, returns are assumed to be normally distributed with constant drift $\mu$ and volatility $\sigma$. This notebook empirically fits these parameters to historical SPY returns and compares the resulting Gaussian distribution to reality.

## Objectives
1. **Fetch historical prices** (cached).
2. **Compute Log Returns**.
3. **Fit Gaussian Model** ($\mu, \sigma$) using Maximum Likelihood (sample mean/std).
4. **Visualize** the fit vs empirical histogram.

In [ ]:
# Setup
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

from qpl.market.data import get_prices
from qpl.market.stats import log_returns, fit_normal_returns

import warnings
warnings.filterwarnings('ignore')

## 1. Data Acquisition

In [ ]:
ticker = "SPY"
start_date = "2020-01-01"
end_date = "2023-12-31"

try:
    df = get_prices(ticker, start=start_date, end=end_date)
    # Handle MultiIndex if present
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    
    prices = df["Close"].values
    dates = df.index
    print(f"Loaded {len(prices)} days for {ticker}")
except Exception as e:
    print(f"Data fetch failed: {e}. Using synthetic.")
    # Fallback synthetic
    prices = 100 * np.exp(np.cumsum(np.random.normal(0.0005, 0.015, size=1000)))
    dates = pd.date_range(start_date, periods=len(prices))
    df = pd.DataFrame({"Close": prices}, index=dates)

## 2. Fit Model Parameters
We calculate annualized $\mu$ and $\sigma$.

In [ ]:
rets = log_returns(prices)

params = fit_normal_returns(rets, annualization=252)

print(f"Fitted Parameters ({start_date} to {end_date}):")
print(f"  Daily Mean (mu_d):   {params.mu_daily:.6f}")
print(f"  Daily Vol  (sig_d):  {params.sigma_daily:.6f}")
print("-" * 30)
print(f"  Annual Mean (mu):    {params.mu_annual:.2%}")
print(f"  Annual Vol  (sigma): {params.sigma_annual:.2%}")

## 3. Distribution Visualization

We overlay the "Theoretical Normal" density (Orange) on the "Empirical" histogram (Blue).

**Observation**: Real market returns often exhibit "Fat Tails" (Kurtosis > 3), meaning extreme events happen more frequently than the Gaussian model predicts. Visually, this appears as a higher peak (more small moves) and fatter tails (more extreme moves) compared to the normal curve.

In [ ]:
plt.figure(figsize=(10, 6))

# 1. Plot Empirical Histogram
count, bins, ignored = plt.hist(rets, bins=60, density=True, alpha=0.6, color='blue', label='Empirical Returns')

# 2. Plot Fitted Gaussian
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 200)
p = stats.norm.pdf(x, params.mu_daily, params.sigma_daily)

# Escape % for mathtext
sigma_str = f"{params.sigma_annual:.0%}".replace("%", "\\%")
plt.plot(x, p, 'r', linewidth=2, label=f'Normal Model ($\sigma={sigma_str}$)')

plt.title(f"{ticker} Returns Distribution vs Fitted Normal")
plt.xlabel("Daily Log Return")
plt.ylabel("Density")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Simple Non-Stationarity Check
If the model were perfect, volatility $\sigma$ would be constant. We plot rolling volatility to visually refute this assumption (Volatility Clustering).

In [ ]:
s_rets = pd.Series(rets, index=dates[1:])
rolling_vol = s_rets.rolling(window=60).std() * np.sqrt(252)

plt.figure(figsize=(10, 4))
plt.plot(rolling_vol.index, rolling_vol, label="60-Day Rolling Volatility", color='purple')
plt.axhline(params.sigma_annual, color='red', linestyle='--', label=f"Constant Fitted Sigma ({params.sigma_annual:.1%})")
plt.title("Volatility Clustering: Empirical vs Constant Model")
plt.ylabel("Annualized Volatility")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()